# Encrypted DataBase Backups in SQL Server

## Master Key

In order to encrypt data bases backups, it's necesary to create a Maste rKey, this Master Key is in charge of protection of all the certificates we need to create.

Creating a MasterKey:

In [ ]:
USE master 
GO 
CREATE MASTER KEY ENCRYPTION BY PASSWORD = 'Master Key Password'


## Certificate

It's possible to have as many certificates as you want/need, the more you have, the safer will be your backup strategy. You can use one certificate to encrypt all your databases backups, or you can create one for each database.

I'll create two certificates to encrypt three databases backups, so, one of them will be used twice.  

If you don't assign an expire date, sql server assumes that it's valid for one year.

In [ ]:
Use Master 
GO

CREATE CERTIFICATE CertificateBaK1
WITH SUBJECT = 'Backups Certificate'
,EXPIRY_DATE = '20201031'; 

CREATE CERTIFICATE CertificadoBK2 
WITH SUBJECT = 'Backup Certificate 2'
,EXPIRY_DATE = '20211006'; 


## Certificates' Backups

Unless you have the .cer and .pvk files, you wan't be able to restore an encrypted backup in a Sql Server Instance different from the one that holds the certificate.

The Master Key and Certificates passwords, should't be the same, but this is up to you.

Last but mot minor point, you must make sure that the sql server agent has access to the foldes that contains the .cer and .pvk files.

In [ ]:
USE master 
GO 
BACKUP CERTIFICATE CertificadteBak  
TO FILE = 'Path\BKPCertificateBak.cer' 
WITH PRIVATE KEY (FILE = 'Path\Backup_Key.pvk' , 
ENCRYPTION BY PASSWORD = 'Certificate1 Password')
GO 

USE master 
GO 
BACKUP CERTIFICATE CertificateBak2
TO FILE = 'Path\BKPCertificateBak2.cer' 
WITH PRIVATE KEY (FILE = 'Path\Backup_Key2.pvk' , 
ENCRYPTION BY PASSWORD = 'Certificate 2 Password')
GO 

## Backups Encrypted

It's time! Let's make an encrypted backup. There are several encryption algorithms but the most recommended is AES\_256.

In [ ]:
BACKUP DATABASE DB1
TO DISK = N'BackupPath\DB1Encrypt.bak' 
 WITH 
 INIT,
 ENCRYPTION 
 ( 
 ALGORITHM = AES_256, 
 SERVER CERTIFICATE = CertificateBak 
 ), 
 COMPRESSION,
 FORMAT,
 STATS = 10 
 GO

BACKUP DATABASE DB2
TO DISK = N'BackupPath\DB2Encrypt.bak' 
 WITH 
 INIT,
 ENCRYPTION 
 ( 
 ALGORITHM = AES_256, 
 SERVER CERTIFICATE = CertificateBak2 
 ), 
 COMPRESSION,
 FORMAT,
 STATS = 10 
 GO

BACKUP DATABASE DB3
TO DISK = N'BackupPath\DB3Encrypt.bak' 
 WITH 
 INIT,
 ENCRYPTION 
 ( 
 ALGORITHM = AES_256, 
 SERVER CERTIFICATE = CertificateBak 
 ), 
 COMPRESSION,
 FORMAT,
 STATS = 10 
 GO

The fact that you have made an encrypted backup, doesn't mean you can't make a regular backup to this database:

In [ ]:
BACKUP DATABASE DB1
TO DISK = N'BackupPath\BD1_Regular.bak' 
 WITH 
 INIT,
 FORMAT,
 COMPRESSION,
 STATS = 10 
 GO

## Certificates in target SQL Server instance

Restoring a database wich backup is encryppted, requires to create the certificate used by this backup in the intance. In order to create it, it is necessary to create a Master Key, in case it doesn't exists already, in the target instance. Then you' ll be able to create the certificate used by the backup, from it's files. 

The Master Key password for that second instance doesn't have to be the same than the one you used in the source instance.

In [ ]:
USE master 
GO 
CREATE MASTER KEY ENCRYPTION BY PASSWORD = 'Master Key Password2'

 CREATE CERTIFICATE CertificateBak 
 FROM FILE = 'TargetPath\CertificateBak.cer'
 WITH PRIVATE KEY (FILE = 'TargetPath\Backup_Key.pvk',  
 DECRYPTION BY PASSWORD = 'Certificate1 Password'); 
 GO  

## DataBase Restore

Now, you can restore the database as you ussually do, either using t-sql or with the wizard ssms.

## Cleanning

Once you finish, you can drop the certificate so it can't be use to make an other restore or backup. If need it, you can create it again.

In [ ]:
DROP CERTIFICATE CertificateBak

# Expyred Certificate

Don't worry, you drop it and create it again with a new expiration date. Of course, you can anticipate it by running a task that check this for you by using the  **sys.certificates** catalog view.

# Good Practices

Keep the certificates' bakups outside the server, as you do with you databases backups.

Do not use the same password neither for different Master Key nor for diferent certificates.